In [1]:
# Auteures : Safae SEBAY - Zara Zulfiqar 
# BUT3 SD EMS FA - G32
# Objectif du notebook : migrer les données sur Mongodb 

In [2]:
import sqlite3
import pymongo
from bson.json_util import dumps

# Connexion à SQLite
sqlite_conn = sqlite3.connect(r"C:/Users/zara_/Downloads/ClassicModel.sqlite")
sqlite_cursor = sqlite_conn.cursor()

# Connexion à MongoDB
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["ClassicModelDB"]


# Migration de la collection Customers qui contient les infos des taables Customers, Orders, OrderDetails, Products et Payments

sqlite_cursor.execute("SELECT * FROM Customers")
customers = sqlite_cursor.fetchall()

for customer in customers:
    customer_doc = {
        "customerNumber": customer[0],
        "customerName": customer[1],
        "contactName": {
            "firstName": customer[3],
            "lastName": customer[2]
        },
        "phone": customer[4],
        "address": {
            "line1": customer[5],
            "line2": customer[6],
            "city": customer[7],
            "state": customer[8],
            "postalCode": customer[9],
            "country": customer[10]
        },
        "orders": [],
        "payments": []
    }

    # Ajouter les commandes
    sqlite_cursor.execute("SELECT * FROM Orders WHERE customerNumber = ?", (customer[0],))
    orders = sqlite_cursor.fetchall()

    for order in orders:
        order_doc = {
            "orderNumber": order[0],
            "orderDate": order[1],
            "requiredDate": order[2],
            "shippedDate": order[3],
            "status": order[4],
            "comments": order[5],
            "orderDetails": []
        }

        # Ajouter les détails des commandes
        sqlite_cursor.execute("SELECT * FROM OrderDetails WHERE orderNumber = ?", (order[0],))
        order_details = sqlite_cursor.fetchall()

        for detail in order_details:
            # Récupérer les informations des produits
            sqlite_cursor.execute("SELECT * FROM Products WHERE productCode = ?", (detail[1],))
            product = sqlite_cursor.fetchone()

            order_doc["orderDetails"].append({
                "productCode": detail[1],
                "productName": product[1],
                "quantityOrdered": detail[2],
                "priceEach": detail[3],
                "buyPrice": product[6],
                "MSRP": product[7]
            })

        customer_doc["orders"].append(order_doc)

    # Ajouter les paiements
    sqlite_cursor.execute("SELECT * FROM Payments WHERE customerNumber = ?", (customer[0],))
    payments = sqlite_cursor.fetchall()

    for payment in payments:
        customer_doc["payments"].append({
            "checkNumber": payment[1],
            "paymentDate": payment[2],
            "amount": payment[3]
        })

    mongo_db["Customers"].insert_one(customer_doc)

# Migration de la collection Employees qui contient les infos des tables Employees et Offices
sqlite_cursor.execute("SELECT * FROM Employees")
employees = sqlite_cursor.fetchall()

for employee in employees:
    # Récupérer les informations de Offices
    sqlite_cursor.execute("SELECT * FROM Offices WHERE officeCode = ?", (employee[5],))
    office = sqlite_cursor.fetchone()

    employee_doc = {
        "employeeNumber": employee[0],
        "firstName": employee[2],
        "lastName": employee[1],
        "jobTitle": employee[7],
        "reportsTo": employee[6],
        "office": {
            "officeCode": office[0],
            "city": office[1],
            "phone": office[2],
            "address": {
                "line1": office[3],
                "line2": office[4],
                "state": office[5],
                "country": office[6],
                "postalCode": office[7]
            },
            "territory": office[8]
        }
    }

    mongo_db["Employees"].insert_one(employee_doc)

# Fermeture des connexions
sqlite_conn.close()
mongo_client.close()